In [1]:
import mediapipe as mp
import cv2 as cv
import math
import numpy as np
from matplotlib.image import imread
import matplotlib.pyplot as plt
import os
import keyboard
import time
import shutil


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
import os
from PIL import Image
from itertools import islice

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:

cap = cv.VideoCapture('abc2.mp4')
with mp_hands.Hands(
    static_image_mode = False,
    max_num_hands = 2,
    model_complexity=0,
    min_detection_confidence=0.3,
    min_tracking_confidence=0.7) as hands:

    while cap.isOpened():

        success, image = cap.read()
        if not success:
            cap.release()
            cv.destroyAllWindows()
        else:
            if cv.waitKey(25) & 0xFF == ord('q'):
                break

            image.flags.writeable = False
            image = image
            cv.imshow('frame', cv.flip(image, 1))
            results = hands.process(image)
            image.flags.writeable = True

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
            cv.imshow('MediaPipe Hands', cv.flip(image, 1))

            frameWidth = cap.get(cv.CAP_PROP_FRAME_WIDTH)
            frameHeight = cap.get(cv.CAP_PROP_FRAME_HEIGHT)

    cap.release()
    cv.destroyAllWindows()

I0000 00:00:1715812421.677122  157673 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1715812421.682743  159716 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715812421.684571  159716 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2024-05-16 00:33:41.790 Python[32577:157673] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase

In [5]:
def make_folders(actions, no_sequences, DATA_PATH):
    for action in actions: 
        for frame_num in range(no_sequences):
            try: 
                os.makedirs(os.path.join(DATA_PATH, action, str(frame_num)))
            except:
                pass

In [6]:
def extract_keypoints(results, width, height):
    keypoint_values = []
    hand_no = 0

    # as many results as hands
    # as many times through loop as hands
    if results.multi_hand_landmarks:
        for hand_no, handLandmarks in enumerate(results.multi_hand_landmarks):
            hand_no =+ 1
            for point in mp.solutions.hands.HandLandmark:
                # by default, landmarks are returned in their normalized format
                normalizedLandmark = handLandmarks.landmark[point]
                # function returns tuple of x and y coordinates 
                pixelCoordinatesLandmark = mp.solutions.drawing_utils._normalized_to_pixel_coordinates(normalizedLandmark.x, normalizedLandmark.y, width, height)
                keypoint_values.append([normalizedLandmark.x, normalizedLandmark.y, handLandmarks.landmark[point].z])
                keypoint_array = np.array(keypoint_values)
                keypoint_array_flat = keypoint_array.flatten()
        
        if (hand_no == 1) and (len(keypoint_array_flat) < 126): 
            zero_array = np.zeros(63)
            if results.multi_handedness[0].classification[0].label == 'Right':
                keypoint_array_flat = np.append(keypoint_array_flat, zero_array)
            elif results.multi_handedness[0].classification[0].label == 'Left':
                keypoint_array_flat = np.append(zero_array, keypoint_array_flat)

        return (keypoint_array_flat)

In [7]:
DATA_PATH = os.path.join('MP_Data_test') 
actions = [ 'hola']
key = [1.0]
key_action = dict(zip(key,actions))
no_sequences = 15
sequence_length = 20

make_folders(actions, no_sequences, DATA_PATH)

In [9]:
cap = cv.VideoCapture('abc2.mp4')
with mp_hands.Hands(
    static_image_mode = False,
    max_num_hands =2 ,
    model_complexity=0,
    min_detection_confidence=0.3,
    min_tracking_confidence=0.7) as hands:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print('error')
            cap.release()
            cv.destroyAllWindows()
        else:
            if cv.waitKey(5) & 0xFF == ord('q'):
                break
            image.flags.writeable = False
            image = image
            cv.imshow('frame', cv.flip(image, 1))
            results = hands.process(image)
            image.flags.writeable = True

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
            cv.imshow('MediaPipe Hands', cv.flip(image, 1))
            frameWidth = cap.get(cv.CAP_PROP_FRAME_WIDTH)
            frameHeight = cap.get(cv.CAP_PROP_FRAME_HEIGHT)
            number = 1
            count = 2
            if keyboard.is_pressed(str(number)):
                print('keyboard pressed: {}'.format(number))
                for frame_num in range(sequence_length):
                    #cv.imwrite('Frame'+str(frame_num)+'.jpg', image)
                    keypoints = extract_keypoints(results, frameWidth, frameHeight)
                    action = key_action[number]
                    npy_path = os.path.join(DATA_PATH, action, str(count), str(frame_num))
                    print(os.path.join(DATA_PATH, action, str(count), str(frame_num)))
                    np.save(npy_path, keypoints)
                    success, image = cap.read()
                    frame_num = frame_num +1

    cap.release()
    cv.destroyAllWindows()

I0000 00:00:1715812510.126980  157673 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1715812510.134426  160575 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715812510.138991  160575 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


ValueError: ("Key '1' is not mapped to any known key.", ValueError('Unrecognized character: 1'))

In [14]:
results.multi_hand_landmarks

In [15]:
array_results = np.load('MP_Data/c/0/9.npy', allow_pickle=True)
print(np.array(array_results).shape)

(126,)
